# 🏁 5주차 미션
- Pytorch를 이용해 직접 이미지 데이터를 간단한 모델로 학습해보는 실습을 진행해보도록 하겠습니다.
- MNIST는 손글씨 데이터셋으로 0~9까지의 숫자를 분류하는 문제입니다.
- 각 단계별 구현 방법은 중요한 내용이기 때문에 하나씩 구현해보세요.

### 📌Q1. 가장 먼저 학습 데이터를 준비해보도록 하겠습니다. MNIST 데이터셋을 직접 Load해봅시다. 데이터셋을 로드하고 DataLoader를 구현해보세요.
- DataLoader를 이용해 MNIST 데이터셋을 로드해봅시다.

```python
# 코드 예제
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

training_epochs = 15 # traning 반복 횟수
batch_size = 100

root = './data'
mnist_train = dset.MNIST(root=root, train=True, transform=transform, download=True)
mnist_test = dset.MNIST(root=root, train=False, transform=transform, download=True)

# data loder를 직접 구현해보자.
train_loader = # 구현
test_loader = # 구현
```

In [2]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

training_epochs = 15 # traning 반복 횟수
batch_size = 100 # 보통 batch size는 2의 배수 단위로 설정 (컴퓨터의 2진 단위 데이터 전송 때문)

root = './data'
# transforms.ToTensor()를 통해, data의 타입을 Tensor 형태로 쉽게 변환 가능
mnist_train = dset.MNIST(root=root, train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dset.MNIST(root=root, train=False, transform=transforms.ToTensor(), download=True)

# data loder를 직접 구현해보자.
train_loader = DataLoader(
    mnist_train, # 위에서 정의한 학습용 MNIST dataset
    batch_size = batch_size, # data를 받아오는 단위인 batch_size 설정
    shuffle = True, # 학습 순서 무작위로 섞기
    drop_last = True # batch_size 단위로 data를 받아올 수 없는 경우 마지막 데이터는 버린다.
)
test_loader = DataLoader(
    mnist_test, # 위에서 정의한 검증용 MNIST dataset
    batch_size = batch_size,  
    shuffle = False, # 검증할 때는 순서를 섞지 않는다.
)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [3]:
print(mnist_train)
print(mnist_test)

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: ToTensor()
Dataset MNIST
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: ToTensor()


In [4]:
print(train_loader)
print(test_loader)

### 📌Q2. 데이터가 준비 되었다면, 이제 그 데이터를 학습할 모델을 구현할 차례입니다. 그 후 모델 안의 가중치를 초기화시켜보세요. 입력 데이터 형태에 맞도록 linear한 모델을 구성해보세요.
- MNIST 입력의 크기는 28x28입니다.
- 여기서 구현하는 linear 모델은 입력이 1차원이기 때문에 입력 차원을 맞춰보세요.

```python
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
linear = # 구현
# weight init
```

In [5]:
# GPU를 사용 가능할 경우, device에 'cuda'를 사용하도록 명시
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
linear = nn.Linear(
    in_features = 28*28, # MNIST 입력 포맷: 28x28 = 784
    out_features = 10 # MNIST label의 포맷: 0~9까지의 숫자
    )

# 가중치 초기화
# 평균이 0이고, 표준편차가 1이 되도록 표준 정규 분포로 초기화
linear.weight.data.normal_(mean=0.0, std=1.0)
print(linear.weight.data)
print(f'Mean: {linear.weight.data.mean()}')
print(f'Std: {linear.weight.data.std()}')

tensor([[-1.1915,  1.3020, -1.1805,  ...,  0.5777,  1.7988, -0.9239],
        [ 0.9808, -0.2161,  1.7329,  ..., -0.6160, -0.1929, -0.4549],
        [-0.6816,  1.0100, -0.3637,  ...,  1.4000, -0.2851, -0.3455],
        ...,
        [-0.1441, -2.3945, -1.5977,  ...,  0.3560, -1.8190,  0.0631],
        [ 0.2929, -0.3353,  0.0384,  ..., -0.6935, -0.8259,  0.3398],
        [-0.1684, -1.1390, -0.7272,  ..., -0.8310, -1.5586,  0.8902]])
Mean: 0.01698700711131096
Std: 0.9937076568603516


In [6]:
print(linear)

Linear(in_features=784, out_features=10, bias=True)


### 📌Q3. 위에서 구현한 모델을 학습시키기 위해서는 loss 함수와 optimizer가 필요합니다. 아래 제시된 loss 함수와 optimizer를 구현해보세요. Loss 함수와 optimizer는 모델 안의 가중치를 업데이트 할 때 사용됩니다.
- 옵티마이저는 SGD, Loss는 Cross Entropy Loss를 사용합니다.

```python
# Loss ftn - Cross Entropy Loss
criterion = # 구현

# Optimizer - SGD
optimizer = # 구현
```

In [7]:
# Loss ftn - Cross Entropy Loss
criterion = nn.CrossEntropyLoss().to(device)

# Optimizer - SGD
learning_rate = 0.1 # 학습률
optimizer = torch.optim.SGD(linear.parameters(), lr=learning_rate)

In [8]:
print(criterion)
print(optimizer)

CrossEntropyLoss()
SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.1
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


### 📌Q4. 3번 문제까지 해결하셨다면, 이제 학습을 위한 준비는 거의 끝났다고 볼 수 있습니다. 위 구현 함수들을 이용해 학습 Loop를 구현해보세요.
- 위에서 구현한 모델, optimizer, loss function 등을 이용해 학습을 구현해주세요.

```python
for epoch in range(training_epochs):
    for i, (imgs, labels) in enumerate(train_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        imgs = imgs.view(-1, 28*28)

        outputs = # 구현
        loss = # 구현

        optimizer. # optimizer zero grad 구현
        loss. # loss backward 구현
        optimizer. # optimizer step 구현

        _, argmax = torch.max(outputs, 1)
        accuracy = (labels == argmax).float().mean()

        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss {:.4f}, Accuracy: {:.2f}%'.format
            (epoch+1, training_epochs, i+1, len(train_loader), loss.item(0, accuracy.item()*100))
```

In [9]:
linear.to(device)
next(linear.parameters()).get_device()

-1

In [ ]:
# 위에서 설정한 training_epoch 만큼 for loop을 돌면서, 학습 수행
for epoch in range(training_epochs):
    # 학습에 사용할 img data와 loss를 계산할 때 사용할 label을 dataloader로부터 받아옴
    for i, (imgs, labels) in enumerate(train_loader):
        # 받아온 data를 각 장치에 보내고 변수에 넣는다.
        imgs, labels = imgs.to(device), labels.to(device)
        # MNIST 이미지는 2차원이므로, Linear 모델에 적합하도록 1차원으로 풀어줌
        imgs = imgs.view(-1, 28*28)

        # 앞서 선언한 linear 모델에 train 데이터를 입력
        # print(imgs.get_device())
        # print(next(linear.parameters()).get_device())
        outputs = linear(imgs)

        # 위에서 설정한 Cross Entropy Loss에 모델의 출력 값과 정답 값을 넣어줌
        loss = criterion(outputs, labels)

        # loop를 돌고 난 후 역전파 과정에서 gradient가 누적되지 않도록 zero_grad() 사용
        optimizer.zero_grad() 
        # loss를 통해 각 node의 gradient를 역전파를 통해서 구함
        loss.backward() 
        # 계산된 gradient를 통해 각 weight, bias를 업데이트
        optimizer.step()

        # 출력이 가장 큰 data의 index 반환
        _, argmax = torch.max(outputs, 1)
        # 값이 같으면 1, 같지 않으면 0
        # 제대로 분류된 개수의 평균 = 정확도
        accuracy = (labels == argmax).float().mean()

        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss {:.4f}, Accuracy: {:.2f}%'.format(
                epoch+1, training_epochs, i+1, len(train_loader), loss.item(), accuracy.item()*100))

Epoch [1/15], Step [100/600], Loss 3.2822, Accuracy: 53.00%
Epoch [1/15], Step [200/600], Loss 2.4761, Accuracy: 58.00%
Epoch [1/15], Step [300/600], Loss 1.4987, Accuracy: 71.00%
Epoch [1/15], Step [400/600], Loss 1.1577, Accuracy: 73.00%
Epoch [1/15], Step [500/600], Loss 1.3212, Accuracy: 75.00%
Epoch [1/15], Step [600/600], Loss 0.9560, Accuracy: 79.00%
Epoch [2/15], Step [100/600], Loss 1.2269, Accuracy: 80.00%
Epoch [2/15], Step [200/600], Loss 0.7297, Accuracy: 86.00%
Epoch [2/15], Step [300/600], Loss 0.8456, Accuracy: 84.00%
Epoch [2/15], Step [400/600], Loss 1.0409, Accuracy: 84.00%
Epoch [2/15], Step [500/600], Loss 0.4868, Accuracy: 91.00%
Epoch [2/15], Step [600/600], Loss 0.5946, Accuracy: 82.00%
Epoch [3/15], Step [100/600], Loss 0.7985, Accuracy: 81.00%
Epoch [3/15], Step [200/600], Loss 0.7393, Accuracy: 81.00%
Epoch [3/15], Step [300/600], Loss 0.7559, Accuracy: 86.00%
Epoch [3/15], Step [400/600], Loss 1.0810, Accuracy: 79.00%
Epoch [3/15], Step [500/600], Loss 0.857

### 📌Q5. 학습이 완료되면, 모델이 잘 동작하는지 테스트가 필요합니다. 데이터로드 파트에서 준비했던 테스트 데이터를 이용해 테스트를 진행해봅시다. 아래 테스트 코드를 완성해보세요.

```python
linear.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i, (img, labels) in enumerate(test_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        imgs = imgs.view(-1, 28*28)

        outputs = # 구현

        _, argmax = torch.max(outputs, 1) # max()를 통해 최종 출력이 가장 높은 class 선택
        total += imgs.size(0)
        correct += (labels == argmax).sum().item()

        print('Test accuracy for {} images: {:.2f}%'.format(total, correct/total*100))
```

In [ ]:
# 각 모델의 평가 과정에서 필요하지 않은 layer를 off 하는 함수
linear.eval()
# 테스트의 경우, 가중치의 업데이트가 되어선 안되므로 no_grad() 사용
with torch.no_grad():
    # 정확도를 계산하기 위한 total, correct 값 0으로 초기화
    correct = 0
    total = 0
    # 위에서 설정한 test DataLoader에서 값을 받아옴
    for i, (imgs, labels) in enumerate(test_loader):
        # 각 장치로 전달
        imgs, labels = imgs.to(device), labels.to(device)
        # 모델 입력에 맞도록 1차원으로 변환
        imgs = imgs.view(-1, 28*28)

        # 모델에 test 데이터 넣어줌
        outputs = linear(imgs)

        _, argmax = torch.max(outputs, 1) # max()를 통해 최종 출력이 가장 높은 class 선택
        # batch_size 만큼 total에 더함
        total += imgs.size(0)
        # 분류를 제대로 한 경우는 correct 변수에 더함
        correct += (labels == argmax).sum().item()

        # 정확도 출력
        print('Test accuracy for {} images: {:.2f}%'.format(total, correct/total*100))